1. Setup and Initial Data Loading

    Environment Setup: Ensure all necessary Python libraries are installed (geopandas, rasterio, rioxarray, numpy, skfuzzy, etc.).
    Load Grassland Boundary Data: Use geopandas to load the grassland boundary data from a shapefile or other geospatial file format.

2. Data Acquisition and Preprocessing

    Identify and Acquire Datasets: Based on the grassland boundaries, identify the relevant datasets (soil, elevation, climate) you need.
    Data Preprocessing:
        Spatial Alignment: Align all raster datasets to the same spatial extent and resolution, ensuring they match the grassland boundary.
        Data Cleaning: Handle any missing or anomalous data in your datasets.

3. Fuzzy Logic Model Setup

    Define Fuzzy Variables: Create fuzzy variables (Antecedents and Consequents) for each environmental factor (e.g., soil pH, elevation) and the output (e.g., habitat suitability).
    Define Membership Functions: Assign membership functions (e.g., triangular, trapezoidal) to each fuzzy set within your fuzzy variables.

4. Rule Definition

    Create Fuzzy Rules: Define the fuzzy logic rules that relate your input variables (environmental factors) to your output variable (habitat suitability).

5. Model Integration and Execution

    Integrate Spatial Data with Fuzzy Model:
        For each pixel or spatial unit in your aligned datasets, extract the necessary values (e.g., soil pH, elevation).
        Input these values into your fuzzy logic model to compute habitat suitability.
    Run the Model: Use the fuzzy logic system to process the data and output habitat suitability scores.

6. Post-processing and Visualization

    Generate Output Raster: Convert the habitat suitability scores back into a raster format, aligning it with your original spatial data.
    Visualization: Use tools like matplotlib to visualize the habitat suitability across the grassland area.

7. Analysis and Interpretation

    Interpret Results: Analyze the habitat suitability map to draw conclusions about the most suitable areas for Sorghastrum nutans.
    Validation (Optional): If validation data is available, compare your model's output to assess its accuracy.

8. Documentation and Reporting

    Document the Workflow: Ensure that each step of your process is well-documented.
    Prepare a Final Report: Summarize your methodology, findings, and any conclusions or recommendations.

Usage of Grassland Boundary

    Initial Data Filtering: Use the grassland boundary to filter or clip your environmental datasets right after loading them. This ensures that all subsequent analyses focus only on the relevant areas.
    Spatial Analysis: In the model integration step, use the boundary to ensure that habitat suitability is only calculated for areas within the grassland.

This structure provides a comprehensive approach to building and executing your fuzzy logic model, ensuring that spatial considerations are integrated effectively with fuzzy logic principles.

In [15]:
%%bash
# Create a virtual environment
python -m venv venv

# The activation command differs between Windows and Unix/MacOS, so choose accordingly




Windows Subsystem for Linux has no installed distributions.
Distributions can be installed by visiting the Microsoft Store:
https://aka.ms/wslstore


CalledProcessError: Command 'b'# Create a virtual environment\npython -m venv venv\n\n# The activation command differs between Windows and Unix/MacOS, so choose accordingly\n\n\n'' returned non-zero exit status 1.

In [ ]:
# For Unix or MacOS:
source venv/bin/activate

In [12]:
# For Windows (use in a separate cell if you're on Windows):
!venv\Scripts\activate


The system cannot find the path specified.


In [11]:
# Set Envionmentals

# Imports
import os
from glob import glob
import pathlib
import json
import requests
from math import floor, ceil
#Third Party
import earthpy as et
import earthpy.appeears as etapp
# import earthpy.earthexplorer as 
import geopandas as gpd
import hvplot.xarray
import numpy as np
import rioxarray as rxr
import rioxarray.merge as rxrmerge
from shapely.geometry import mapping
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import xarray as xr

# Inlcude rest of modules as needed
with open('settings.json', 'r') as file:
    config = json.load(file)

fuzzy_config = config['fuzzy_logic']
# # Set data directory globals here
# THIS CAN BE DONE THROUGH A JSON ONLY NEED TO LOOP
# THROUGH TO  CHECK IF FILES EXIST

# data_dir = ('/workspaces/golden-feather-grass-habitats/Assets/')
# grassland_path = os.path.join(data_dir, 'study-site-boundaries')
# raster_bin = os.path.join(data_dir, 'bin', 'rasters')
# soil_dir = os.path.join(raster_bin, 'soil')
# climate_dir = os.path.join(raster_bin, 'climate')
# dem_dir = os.path.join(raster_bin, 'DEM')
# topo_dir = os.path.join(raster_bin, 'topo-derive')
# vectors_dir = os.path.join(data_dir, 'vector')

# directories = [grassland_path, 
#                raster_bin, 
#                soil_dir, 
#                climate_dir, 
#                dem_dir, 
#                topo_dir, 
#                vectors_dir]

# for a_dir in (directories):
#     if not os.path.exists(a_dir):
#         os.makedirs(a_dir)

ModuleNotFoundError: No module named 'earthpy'

In [ ]:
# Get boundary gdf for clipping

# Download and load boundary data
boundary_config = config['boundary']
boundary_gdf = gpd.read_file(boundary_config['url'])
boundary_gdf = boundary_gdf.to_crs(boundary_config['crs'])

boundary_gdf


study_site_gdf = (boundary_gdf
                  .set_index('GRASSLANDN')
                  .loc[[config['study_site']]]
                  )
study_site_gdf

# Make sure to check crs 

# Can run through study site gdf as loop later

#Plot code here to test geometry

,NATIONALGR,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry
GRASSLANDN,,,,,
Caddo National Grassland,295515010328,68479.549,0.02694,1.159342,"MULTIPOLYGON (((-95.85492 33.79814, -95.85494 ..."


In [ ]:


def download_soil_data(study_site_gdf, config):
    raster_dir = ("/workspaces/""golden-feather-grass-habitats/"
                  "assets/bin/rasters/soil")
    os.makedirs(raster_dir, exist_ok=True)

    bounds = study_site_gdf.total_bounds
    min_lon, min_lat, max_lon, max_lat = bounds

    min_lat, min_lon = floor(min_lat), floor(min_lon)
    max_lat, max_lon = ceil(max_lat), ceil(max_lon)

    soil = config['data_sources']['soil']
    property = soil['property']
    stat = soil['stat']
    depth = soil['depth']

    rasters = []
    for lat in range(min_lat, max_lat):
        for lon in range(min_lon, max_lon):
            polaris_url = (
                "http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/"
                "{property}/{stat}/{depth}/"
                "lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif").format(
                    property=property, 
                    stat=stat, 
                    depth=depth,
                    min_lat=lat,
                    min_lon=lon,
                    max_lat=min(lat + 1, max_lat),
                    max_lon=min(lon + 1, max_lon))

            local_path = os.path.join(raster_dir, f"soil_data_{lat}_{lon}.tif")
            response = requests.get(polaris_url)
            if response.status_code == 200:
                with open(local_path, 'wb') as file:
                    file.write(response.content)
                rasters.append(local_path)

    # Merge rasters into a single data array
    merged_raster = None
    for raster in rasters:
        raster_data = rxr.open_rasterio(raster, masked=True).squeeze()
        if merged_raster is None:
            merged_raster = raster_data
        else:
            merged_raster = rxrmerge.merge_arrays([merged_raster, raster_data])

    # Reproject the raster to match the CRS of the study site GDF if they are different
    if merged_raster.rio.crs != study_site_gdf.crs:
        merged_raster = merged_raster.rio.reproject(study_site_gdf.crs)

    # Clip the raster to the study site boundary
    clipped_raster = merged_raster.rio.clip_box(*study_site_gdf.total_bounds)

    return clipped_raster


soil_data = download_soil_data(study_site_gdf, config)
soil_data

<xarray.DataArray (y: 1500, x: 1133)>
array([[6.2812977, 6.0364027, 6.2213516, ..., 5.0641117, 5.0497384,
        5.071565 ],
       [6.0364027, 7.7850313, 6.3247724, ..., 5.0692344, 5.0598965,
        5.0557575],
       [7.7850313, 6.2865696, 6.4395437, ..., 5.051176 , 5.0498123,
        5.057178 ],
       ...,
       [7.7804494, 7.7679377, 7.795513 , ..., 7.448288 , 7.2402515,
        7.289378 ],
       [7.7898827, 7.8332634, 7.841543 , ..., 7.3668976, 7.223908 ,
        7.5178947],
       [7.8002653, 7.785071 , 7.826631 , ..., 7.2944937, 7.439492 ,
        7.510687 ]], dtype=float32)
Coordinates:
  * x            (x) float64 -96.17 -96.17 -96.17 ... -95.86 -95.86 -95.85
  * y            (y) float64 33.8 33.8 33.8 33.8 ... 33.39 33.39 33.39 33.39
    band         int64 1
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     nan

In [ ]:
#Define import functions here
# Check if arrays or downloads already exist for each layer,  if not 
# download and create array
# Merge arrays if necessary

# elevation <-- DEM from appears
study_site_name = config['study_site']
download_key = (study_site_name
    .replace("National Grassland", "SRTM")
    .replace(" ", "-"))
srtm_dir = config['data_sources']['elevation']['local_path']

srtm_downloader = etapp.AppeearsDownloader(
    download_key = download_key,
    ea_dir = srtm_dir,
    product='SRTMGL1_NC.003',
    layer = 'SRTMGL1_DEM',
    start_date='02-11-2000',
    end_date='02-21-2000',
    polygon= study_site_gdf)
if not os.path.exists(srtm_downloader.data_dir):
    srtm_downloader.download_files()

# srtm_downloader
srtm_paths = glob(
    os.path.join(
        srtm_downloader.data_dir,
         'SRTMGL1_NC.003*',
          '*.tif' ))

print(srtm_paths)
[rxr.open_rasterio(srtm_path, masked=True).squeeze() for srtm_path in srtm_paths][0]
 
# def dem_downloader(x1):
# climate <--- check elsa's post
# derived topo <-- ask chatgpt + check old project 

['/workspaces/golden-feather-grass-habitats/assets/bin/rasters/DEM/Caddo-SRTM/SRTMGL1_NC.003_2000001_to_2023344/SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0001.tif']


<xarray.DataArray (y: 1500, x: 1132)>
[1698000 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 -96.17 -96.17 -96.17 ... -95.86 -95.86 -95.86
  * y            (y) float64 33.8 33.8 33.8 33.8 ... 33.39 33.39 33.39 33.39
    spatial_ref  int64 0
Attributes:
    add_offset:     0.0
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    units:          Meters

In [ ]:
maca_url = ('http://thredds.northwestknowledge.net:8080/thredds/ncss/'
            'agg_macav2metdata_pr_bcc-csm1-1-m_r1i1p1_historical_1950_2005_CONUS_monthly.nc'
            '?var=precipitation'
            '&disableLLSubset=on&disableProjSubset=on'
            '&horizStride=1'
            '&time_start=1950-01-15T00%3A00%3A00Z&time_end=1950-12-15T00%3A00%3A00Z'
            '&timeStride=1&accept=netcdf')
maca_response = requests.get(maca_url)

# Grab the directory path from json
climate_path = config['data_sources']['climate']['local_path']

# Create the directory if it doesn't exist
if not os.path.exists(climate_path):
    os.makedirs(climate_path, exist_ok=True)

# Define the full path including the filename
maca_path = os.path.join(climate_path, 'maca.nc')

# Assuming maca_response is obtained from a requests.get() call
maca_response = requests.get(maca_url)

# Write the file to the specified directory
with open(maca_path, 'wb') as maca_file:
    maca_file.write(maca_response.content)

In [ ]:
maca_ds = xr.open_dataset(maca_path, engine = 'netcdf4')
maca_ds = maca_ds.assign_coords(lon=maca_ds.lon - 360)
maca_ds = maca_ds.rio.write_crs(4326)
precip_da = maca_ds.precipitation
precip_da.rio.write_crs(4326, inplace = True)
precip_da.rio.set_spatial_dims('lon','lat', inplace = True)

maca_ds.precipitation.mean('time').hvplot(rasterize=True)


BokehModel(combine_events=True, render_bundle={'docs_json': {'4e83462d-b3c5-4f2d-bd78-c31a3a6b3db7': {'version…

Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Callback.process_on_change() done, defined at /opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py:355> exception=UnsetValueError("figure(id='p1016', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 374, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 281, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/opt/conda/lib/python3.10/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValueError: figure(id='p1016', ...).inner_height doesn't have a value set

During handling of the above exception, an

In [ ]:
precip_da.rio.clip_box(*study_site_gdf.total_bounds).mean('time').hvplot()

:Image   [lon,lat]   (precipitation)

In [ ]:
#Prune and Harmonization functions here
# Check if all arrays have same crs, area, and bounds
# throw error if not, print values to compare
# get rid of unneccesary layers






In [ ]:
# Function to create fuzzy sets
def create_fuzzy_sets(variable, sets_config):
    for set_name, set_details in sets_config.items():
        if set_details['type'] == 'triangular':
            variable[set_name] = fuzz.trimf(variable.universe, set_details['params'])

# Create fuzzy variables
variables = {}
for var_name, var_details in fuzzy_config['variables'].items():
    range_min, range_max, range_step = var_details['range']
    antecedent = ctrl.Antecedent(np.arange(range_min, range_max, range_step), var_name)
    create_fuzzy_sets(antecedent, var_details['sets'])
    variables[var_name] = antecedent

# Create output variable for habitat suitability
habitat_suitability = ctrl.Consequent(np.arange(0, 100, 1), 'habitat_suitability')


In [ ]:
# Define rules based on JSON
rules = []
for rule in fuzzy_config['rules']:
    condition = None
    for cond in rule['if']:
        if cond == "AND":
            continue
        var_name, set_name = cond.split('.')
        if condition is None:
            condition = variables[var_name][set_name]
        else:
            condition = condition & variables[var_name][set_name] if "AND" in rule['if'] else condition | variables[var_name][set_name]
    consequent = habitat_suitability[rule['then']]
    rules.append(ctrl.Rule(condition, consequent))

# Control system
habitat_ctrl = ctrl.ControlSystem(rules)
habitat_simulation = ctrl.ControlSystemSimulation(habitat_ctrl)


In [ ]:
# start building fuzzy logic model here 

# Example: Defining fuzzy variables and sets
soil_pH = ctrl.Antecedent(np.arange(0, 14, 1), 'soil_pH')
elevation = ctrl.Antecedent(np.arange(0, 3000, 1), 'elevation')  # Example range
precipitation = ctrl.Antecedent(np.arange(0, 500, 1), 'precipitation')  # Example range
habitat_suitability = ctrl.Consequent(np.arange(0, 100, 1), 'habitat_suitability')

# Define membership functions
soil_pH['acidic'] = fuzz.trimf(soil_pH.universe, [0, 0, 5.5])
# ... continue for other sets

# Define rules
rule1 = ctrl.Rule(soil_pH['neutral'] & elevation['medium'] & precipitation['moderate'], habitat_suitability['high'])
# ... more rules

# Control system
habitat_ctrl = ctrl.ControlSystem([rule1, ...])
habitat_simulation = ctrl.ControlSystemSimulation(habitat_ctrl)

# Applying the model to data
# This would involve iterating over your raster data, applying the model to each pixel
# Example:
habitat_simulation.input['soil_pH'] = 6.5  # Example input
# ... set other inputs
habitat_simulation.compute()
print(habitat_simulation.output['habitat_suitability'])

# Visualization code goes here